<a href="https://colab.research.google.com/github/NaimiMariem/QR_JP_Morgan_Chase/blob/main/Task_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# Ce texte est au format code
```
<div>
  <h1 style="font-size: 24px; font-weight: bold;">Bucket FICO scores</h1>

<p>To construct a rating map that maps FICO scores of borrowers to ratings (where a lower rating signifies a better credit score), we need to perform quantization, which involves dividing the range of FICO scores into specific buckets or intervals. We can explore two approaches for quantization: Mean Squared Error (MSE) and Log-Likelihood.
The method we chose in this approach is the log-likelihood method which requires dynamic programming.

In [13]:
import numpy as np
import pandas as pd

raw_url="https://raw.githubusercontent.com/NaimiMariem/QR_JP_Morgan_Chase/main/Task%203%20and%204_Loan_Data%20(1).csv"

df = pd.read_csv(raw_url)
fico_scores= df['fico_score'].values

In [14]:
def quantization_log_likelihood(fico_scores, num_buckets):
    # Sort the FICO scores in ascending order
    sorted_fico_scores = np.sort(fico_scores)

    # Calculate the number of FICO scores per bucket
    scores_per_bucket = len(sorted_fico_scores) // num_buckets

    # Initialize the dynamic programming table to store the log-likelihood values
    dp_table = np.zeros((num_buckets + 1, len(sorted_fico_scores) + 1))
    dp_table[0, :] = 0

    # Define a function to compute the log-likelihood for a given bucket
    def log_likelihood(boundary_idx, start_idx, end_idx):
        num_records = end_idx - start_idx
        num_defaults = np.sum(sorted_fico_scores[start_idx:end_idx])
        prob_default = num_defaults / num_records
        return num_defaults * np.log(prob_default) + (num_records - num_defaults) * np.log(1 - prob_default)

    # Perform dynamic programming to fill the dp_table
    for i in range(1, num_buckets + 1):
        for j in range(i * scores_per_bucket, len(sorted_fico_scores) + 1):

            dp_table[i, j] = max(dp_table[i, j - 1], max(dp_table[i - 1, k] + log_likelihood(i, k, j)
                                 for k in range((i - 1) * scores_per_bucket, j)))

    # Backtrack to find the optimal bucket boundaries that maximize the log-likelihood
    bucket_boundaries = []
    start_idx = len(sorted_fico_scores)
    for i in range(num_buckets, 0, -1):
        for k in range(start_idx - 1, (i - 1) * scores_per_bucket - 1, -1):
            if dp_table[i, start_idx] == dp_table[i - 1, k] + log_likelihood(i, k, start_idx):
                bucket_boundaries.append(sorted_fico_scores[k])
                start_idx = k
                break

    # Create the rating map (bucket boundaries and corresponding values)
    rating_map = {}
    rating = 1  # Assign ratings in reverse order (higher rating to lower FICO score)
    for boundary in reversed(bucket_boundaries):
        rating_map[boundary] = rating
        rating += 1

    return rating_map

In [ ]:
#Example
num_buckets=4
rating_map= quantization_log_likelihood(fico_scores, num_buckets)
return(rating_map)

<ipython-input-14-d511d2d71e4f>:17: RuntimeWarning: invalid value encountered in log
  return num_defaults * np.log(prob_default) + (num_records - num_defaults) * np.log(1 - prob_default)
